# SETUP SYNTH ENGINE

In [1]:
from scipy.io import wavfile
import dawdreamer as daw
import numpy as np
import os
SAMPLE_RATE = 44100

BUFFER_SIZE = 128 # Parameters will undergo automation at this buffer/block size.
PPQN = 960 # Pulses per quarter note.
SYNTH_PLUGIN = os.path.expanduser('~')+"/../../Library/Audio/Plug-Ins/VST/TAL-NoiseMaker.vst"#os.path.expanduser('~')+"/../../Library/Audio/Plug-Ins/Components/TAL-NoiseMaker.component"  # extensions: .dll, .vst3, .vst, .component
# REVERB_PLUGIN = "C:/path/to/reverb.dll"  # extensions: .dll, .vst3, .vst, .component
# MIDI_PATH = "../resources/MIDI/song.mid"

engine = daw.RenderEngine(SAMPLE_RATE, BUFFER_SIZE)

# Make a processor and give it the unique name "my_synth", which we use later.
synth = engine.make_plugin_processor("my_synth", SYNTH_PLUGIN)
synth.load_state('../resources/plugins/params/save_states/BasePad')


error: attempt to map invalid URI `/Users/vedant/../../Library/Audio/Plug-Ins/VST/TAL-NoiseMaker.vst'
Attempting to load VST: /Library/Audio/Plug-Ins/VST/TAL-NoiseMaker.vst
Creating VST instance: TAL-NoiseMaker
Initialising VST: TAL-NoiseMaker (5.0.4.0)
Attempting to load VST: /Library/Audio/Plug-Ins/VST/TAL-NoiseMaker.vst
Creating VST instance: TAL-NoiseMaker
Initialising VST: TAL-NoiseMaker (5.0.4.0)


# ADDITIONAL PROCESSING (Not part of pipeline)

In [10]:
import pandas as pd
# master_parameters = pd.DataFrame()
# param_name=[]
# param_number = []
# preset_names=[]
# param_value=[]

# for preset_name in os.listdir('../resources/plugins/params/save_states'):
#     synth.load_state('../resources/plugins/params/save_states/'+preset_name)
#     for i in range(len(synth.get_parameters_description())):
#         param_name.append(synth.get_parameter_name(i).lower().replace(' ','_'))
#         param_number.append(i)
#         preset_names.append(preset_name)
#         param_value.append(synth.get_parameter(i))

# master_parameters['parameter_name'] = param_name
# master_parameters['parameter_number'] = param_number
# master_parameters['preset_name'] = preset_names
# master_parameters['parameter_value'] = param_value

# master_parameters['mean'] = master_parameters['parameter_value']
# master_parameters['std'] = master_parameters['parameter_value']/10

master_parameters = pd.read_csv('../resources/working_data/preset_parameter_mapping.csv', header=0)

# CREATE MIDI NOTES/CHORD MAPPING

In [ ]:
# TODO: Create a mapping between emotion and notes and chords, to be sampled for triggering the synth


# Sampling parameter values for synth timbre

In [14]:
ignored_params = ['-','master_volume','midi_clear', 'midi_lock']

# TODO: create logic mapping presets to emotions
synth_timbre = 'BasePad' # possible values are ['TwoFlower','Bellomatism','GatedPad','FloatingPad','FantasyPad','FuzzyChorda','BigTrancer','LuxuryStrings','BitOrchestra','DarkAnimations','SunRise','DreamOf86','ParallaxPad','PadWobbler','SweetAnalogue','RingmodMod','BurstingPad','Tron','BionicPad','Lightstorm','OpenSquare','Ringmodded','Glasometer','Easymat','BasePad','SpookyOrgan']

target_preset = master_parameters[master_parameters['preset_name'] == synth_timbre]

# Sampling values from a normal distribution
target_preset['value'] = np.random.normal(target_preset['mean'], target_preset['std'])

# Filtering relevant parameters (zero value params retained)
target_values = {}
for i in range(len(target_preset)):
    if target_preset.iloc[i]['parameter_name'] not in ignored_params:
        target_values[target_preset.iloc[i]['parameter_number']] = target_preset.iloc[i]['value']
        
# Loading the preset closest to the required parameters
synth.load_state(f'../resources/plugins/params/save_states/{synth_timbre}')

# Setting parameters sampled from distribution
for param_num in target_values.keys():
    synth.set_parameter(param_num, target_values[param_num])

/var/folders/hs/h9zhlkvj4b9d3mmzwzsd87vr0000gn/T/ipykernel_9838/1187164427.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_preset['value'] = np.random.normal(target_preset['mean'], target_preset['std'])


# RENDERING AUDIO

In [ ]:
synth.add_midi_note(60, 127, 0.5, .25) # (MIDI note, velocity, start, duration)
engine.load_graph([(synth, [])])
engine.render(5)
output = engine.get_audio()
wavfile.write(f'output.wav', SAMPLE_RATE, output.transpose())